In [ ]:
import matplotlib
import matplotlib.pyplot as plt 

from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D

from math import *
from random import random

π = pi

In [ ]:
matplotlib.style.use(['dark_background', 'bmh'])
%matplotlib widget

Car-trailer diagram (inverted image `res/car-trainer-k.png` available as well):
![car-trailer](res/car-trailer-w.png)

Car-trailer equation:
\begin{align}
\dot x &= s \cos \theta_0 \\
\dot y &= s \sin \theta_0 \\
\dot \theta_0 &= \frac{s}{L} \tan \phi \\
\dot \theta_1 &= \frac{s}{d_1} \sin(\theta_1 - \theta_0)
\end{align}
where $s$: signed speed, $\phi$: negative steering angle,

In [ ]:
class Truck:
    def __init__(self, display=False, figure=None):

        self.W = 1  # car and trailer width, for drawing only
        self.L = 1 * self.W  # car length
        self.d = 4 * self.L  # d_1
        self.s = -0.1  # speed
        self.display = display
        
        self.box = [0, 40, -10, 10]
        if self.display:
            if figure is None:
                plt.close('all') # Close any other truck figures
                self.f = plt.figure(figsize=(10, 5), num='The truck backer-upper', facecolor='none')
                self.ax = self.f.add_axes([0.01, 0.01, 0.98, 0.98], facecolor='black')
                self.ax.axis('equal')
                b = self.box
                self.ax.axis([b[0] - 1, b[1], b[2], b[3]])
                self.ax.set_xticks([]); self.ax.set_yticks([])
                self.ax.axhline(); self.ax.axvline()

            # Passing in a figure allows multiple trucks to be displayed
            else:
                self.f = figure
                self.ax = self.f.axes[0]
                
            self.patches = list()
            

        self.reset()

    def reset(self, ϕ=0):
        self.ϕ = ϕ  # car initial steering angle
        
        # self.θ0 = deg2rad(30)  # car initial direction
        # self.θ1 = deg2rad(-30)  # trailer initial direction
        # self.x, self.y = 20, -5  # initial car coordinates
        
        self.θ0 = (random() - 0.5)  * 2 * π  # -π <= ϑ₀ < π
        self.θ1 = (random() - 0.5) * π / 2 + self.θ0  # -π/4 <= ϑ₁ - ϑ₀ < π/4
        self.x = (random() * .75 + 0.25) * self.box[1]
        self.y = (random() - 0.5) * (self.box[3] - self.box[2])
        
        # If poorly initialise, then re-initialise
        if not self.valid():
            self.reset(ϕ)
        
        # Draw, if display is True
        if self.display: self.draw()
    
    def step(self, ϕ=0, dt=1):
        
        # Check for illegal conditions
        if self.is_jackknifed():
            print('The truck is jackknifed!')
            return
        
        if self.is_offscreen():
            print('The car or trailer is off screen')
            return
        
        self.ϕ = ϕ
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        
        # Perform state update
        self.x += s * cos(θ0) * dt
        self.y += s * sin(θ0) * dt
        self.θ0 += s / L * tan(ϕ) * dt
        self.θ1 += s / d * sin(θ0 - θ1) * dt
        
        return self.state()
    
    def state(self):
        return (self.x, self.y, self.θ0, *self._trailer_xy(), self.θ1)
    
    def _get_atributes(self):
        return (
            self.x, self.y, self.W, self.L, self.d, self.s,
            self.θ0, self.θ1, self.ϕ
        )
    
    def _trailer_xy(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        return x - d * cos(θ1), y - d * sin(θ1)
        
    def is_jackknifed(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        return abs(θ0 - θ1) * 180 / π > 90
    
    def is_offscreen(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        
        x1, y1 = x + 1.5 * L * cos(θ0), y + 1.5 * L * sin(θ0)
        x2, y2 = self._trailer_xy()
        
        b = self.box
        return not (
            b[0] <= x1 <= b[1] and b[2] <= y1 <= b[3] and
            b[0] <= x2 <= b[1] and b[2] <= y2 <= b[3]
        )
        
    def valid(self):
        return not self.is_jackknifed() and not self.is_offscreen()
        
    def draw(self):
        if not self.display: return
        if self.patches: self.clear()
        self._draw_car()
        self._draw_trailer()
        self.f.canvas.draw()
            
    def clear(self):
        for p in self.patches:
            p.remove()
        self.patches = list()
        
    def _draw_car(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        ax = self.ax
        
        x1, y1 = x + L / 2 * cos(θ0), y + L / 2 * sin(θ0)
        bar = Line2D((x, x1), (y, y1), lw=5, color='C2', alpha=0.8)
        ax.add_line(bar)

        car = Rectangle(
            (x1, y1 - W / 2), L, W, color='C2', alpha=0.8, transform=
            matplotlib.transforms.Affine2D().rotate_deg_around(x1, y1, θ0 * 180 / π) +
            ax.transData
        )
        ax.add_patch(car)

        x2, y2 = x1 + L / 2 ** 0.5 * cos(θ0 + π / 4), y1 + L / 2 ** 0.5 * sin(θ0 + π / 4)
        left_wheel = Line2D(
            (x2 - L / 4 * cos(θ0 + ϕ), x2 + L / 4 * cos(θ0 + ϕ)),
            (y2 - L / 4 * sin(θ0 + ϕ), y2 + L / 4 * sin(θ0 + ϕ)),
            lw=3, color='C5', alpha=1)
        ax.add_line(left_wheel)

        x3, y3 = x1 + L / 2 ** 0.5 * cos(π / 4 - θ0), y1 - L / 2 ** 0.5 * sin(π / 4 - θ0)
        right_wheel = Line2D(
            (x3 - L / 4 * cos(θ0 + ϕ), x3 + L / 4 * cos(θ0 + ϕ)),
            (y3 - L / 4 * sin(θ0 + ϕ), y3 + L / 4 * sin(θ0 + ϕ)),
            lw=3, color='C5', alpha=1)
        ax.add_line(right_wheel)
        
        self.patches += [car, bar, left_wheel, right_wheel]
        
    def _draw_trailer(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        ax = self.ax
            
        x, y = x - d * cos(θ1), y - d * sin(θ1) - W / 2
        trailer = Rectangle(
            (x, y), d, W, color='C0', alpha=0.8, transform=
            matplotlib.transforms.Affine2D().rotate_deg_around(x, y + W/2, θ1 * 180 / π) +
            ax.transData
        )
        ax.add_patch(trailer)
        
        self.patches += [trailer]

In [ ]:
truck = Truck(display=True)

In [ ]:
ϕ = π/180 * (10)  # positive left, negative right
truck.step(ϕ)
truck.draw()

print(f"Cab angle: {truck.state()[2] * 180 / π:2f}")
print(f"Trailer angle: {truck.state()[5] * 180 / π:2f}")


In [ ]:
truck.reset()

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD
from tqdm import tqdm

In [ ]:
# Build expert data set

episodes = 10
inputs = list()
outputs = list()
truck = Truck(); episodes = 10_000  # uncomment for creating the data set

for episode in tqdm(range(episodes)):
    
    truck.reset()
    
    while truck.valid():
        initial_state = truck.state()
        ϕ = (random() - 0.5) * π / 2
        inputs.append((ϕ, *initial_state))
        outputs.append(truck.step(ϕ))
        truck.draw()

In [ ]:
len(inputs), len(outputs)

In [ ]:
state_size = 6
steering_size = 1
hidden_units_e = 45

emulator = nn.Sequential(
    nn.Linear(steering_size + state_size, hidden_units_e),
    nn.ReLU(),
    nn.Linear(hidden_units_e, state_size)
)

optimiser_e = SGD(emulator.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [ ]:
tensor_inputs = torch.Tensor(inputs)
tensor_outputs = torch.Tensor(outputs)

In [ ]:
mean = tensor_inputs.mean(0)
std = tensor_inputs.std(0)
tensor_inputs = (tensor_inputs - mean) / std
tensor_outputs = (tensor_outputs - mean[1:]) / std[1:]

In [ ]:
# Split the data into 80:20 for test:train.
test_size = int(len(tensor_inputs) * 0.8)
print(len(tensor_inputs), test_size)

train_inputs = tensor_inputs[:test_size]
train_outputs = tensor_outputs[:test_size]
test_inputs = tensor_inputs[test_size:]
test_outputs = tensor_outputs[test_size:]

In [ ]:
len(train_inputs)

In [ ]:
# Emulator training
cnt = 0
for i in torch.randperm(len(train_inputs)):
    ϕ_state = train_inputs[i]
    next_state_prediction = emulator(ϕ_state)
    
    next_state = train_outputs[i]
    loss = criterion(next_state_prediction, next_state)
    
    optimiser_e.zero_grad()
    loss.backward()
    optimiser_e.step()
    
    if cnt == 0 or (cnt + 1) % 1000 == 0:
        print(f'{cnt + 1:4d} / {len(train_inputs)}, {loss.item():.10f}')
    cnt += 1

In [ ]:
# Test
total_loss = 0
with torch.no_grad():
    for idx, ϕ_state in enumerate(test_inputs):
        next_state_prediction = emulator(ϕ_state)

        next_state = test_outputs[idx]
        total_loss += criterion(next_state_prediction, next_state).item()

ave_test_loss = total_loss/test_size
print(f'Test loss: {ave_test_loss:.10f}')

In [ ]:
# Setup a curriculum for training a controller that starts with easier examples and moves to harder ones
# Each "lesson specifies the xrange, yrange, cabin angle relative to x axis, and trailer angle relative to cabin
# Using https://tifu.github.io/truck_backer_upper/ as a reference for the following parts

import numpy as np


def generate_lessons(start_lesson, end_lesson, n=21):
    '''
    Interpolates between a starting lesson and ending lesson to generate n distint lessons
    '''

    lessons = []
    
    for srange, erange in zip(start_lesson, end_lesson):
        (s0, s1), (e0, e1) = srange, erange
        starts = np.round(np.linspace(s0, e0, n), decimals=2)
        ends = np.round(np.linspace(s1, e1, n), decimals=2)

        lessons.append(list(zip(starts, ends)))

    # Little trick to go from [ [xranges], [yranges], ... ] to [(xrange_1, yrange_1,...), (xrange2, yrange2,...),...]
    return list(zip(*lessons))


start = ((4, 10), (-1, 1), (-10, 10), (0, 0))
end = (( 20, 40 ), ( -10, 10 ), ( -90, 90 ), ( -90, 90 ))
    
lessons = generate_lessons(start, end)

lessons

In [ ]:
S = 1000       # Samples per lesson
K = 1000       # Max number of steps


In [ ]:
# Display 20 random initial position(s) of given lessons
truck = Truck(display=True)
trucks = [truck] + [Truck(display=True, figure=truck.f) for _ in range(19)]
l = 0

In [ ]:
import time

print(f"Showing truck(s) from lesson {l+1}")

(x0, x1), (y0, y1), (truckθ0, truckθ1), (trailerθ0, trailerθ1) = lessons[l]

for truck in trucks:
    truck.θ0 = π / 180 * ((random() * (truckθ1 - truckθ0) + truckθ0)) 
    truck.θ1 = π / 180 * ((random() * (trailerθ1 - trailerθ0) + trailerθ0))
    truck.x = random() * (x1 - x0) + x0
    truck.y = random() * (y1 - y0)+ y0
    truck.draw()

l = ( l + 1) % 21 # Repeatedly running this cell loops through the lessons


In [ ]:
# Controller architecture

controller = nn.Sequential(
    nn.Linear(state_size, hidden_units_c),
    nn.Tanh(),
    nn.Linear(hidden_units_c, 1),
    nn.Tanh()
)



In [ ]:
# Ok now let's try training a controller using these lessons
truck = Truck()

optimiser_c = SGD(controller.parameters(), lr=0.005)
loss = nn.MSELoss()
target = (0, 0, 0) # Target of (trailer x, trailer y, trailer θ)

# Controller architecture
hidden_units_c = 50

# Note this could be much faster if we parallelize stuff...
def use_controller(state, controller, emulator, K):
    for step in range(K):
        ϕ = controller(state)
        with torch.no_grad(): # frozen emulator
            state = emulator(torch.cat(ϕ, state))

    return state

def train_sample(lesson):
    # Set starting position

    (x0, x1), (y0, y1), (truckθ0, truckθ1), (trailerθ0, trailerθ1) = lesson

    truck.θ0 = π / 180 * ((random() * (truckθ1 - truckθ0) + truckθ0)) 
    truck.θ1 = π / 180 * ((random() * (trailerθ1 - trailerθ0) + trailerθ0))
    truck.x = random() * (x1 - x0) + x0
    truck.y = random() * (y1 - y0) + y0

    init_state = torch.Tensor(truck.state())
    final_state = use_controller(state, controller, emulator, K)
    
    # Loss is only calculated with trailer xy and angle
    loss = criterion(final_state[3:], target)
    
    optimiser_e.zero_grad()
    loss.backward()
    optimiser_e.step()


for lesson in lessons:
    for i in range(S):
        train_sample(lesson)
        break
    break
        
